In [1]:
%load_ext autoreload

%autoreload 2
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, roc_curve
from ast import literal_eval

import os
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegressionCV, LinearRegression

import warnings
import numpy as np
from collections import OrderedDict

from lob_data_utils import lob, db_result, overview, roc_results, model
from lob_data_utils.svm_calculation import lob_svm

sns.set_style('whitegrid')
warnings.filterwarnings('ignore')

In [2]:
data_length = 10000 # TODO: not used
stock = '9265'

In [3]:
df, df_test = lob.load_prepared_data(
    stock, data_dir='../queue_imbalance/data/prepared', cv=False, include_test=True, length=None)
print(len(df), len(df_test))

12789 3197


In [4]:
def prepare_df(df):
    df.rename(columns={'Unnamed: 0': 'datetime'}, inplace=True)
    df.drop(columns=['bid', 'ask'], inplace=True)
    df.index = df['datetime']
    df = df.sort_index()
    df['prev_queue_imbalance'] = [None] + df['queue_imbalance'].iloc[0:len(df)-1].values.tolist()
    df.dropna(inplace=True)
    return df
df = prepare_df(df)
df_test = prepare_df(df_test)
df.head()

,datetime,bid_price,ask_price,mid_price,sum_sell_ask,sum_buy_bid,mid_price_indicator,queue_imbalance,prev_queue_imbalance
datetime,,,,,,,,,
2013-09-16 13:54:00,2013-09-16 13:54:00,3356.5,3357.0,3356.75,50.0,909.0,0.0,0.895725,0.308792
2013-09-16 13:55:00,2013-09-16 13:55:00,3356.0,3357.0,3356.50,1253.0,1093.0,1.0,-0.068201,0.895725
2013-09-16 13:56:00,2013-09-16 13:56:00,3358.0,3358.5,3358.25,2362.0,586.0,1.0,-0.602442,-0.068201
2013-09-16 13:57:00,2013-09-16 13:57:00,3358.5,3359.5,3359.00,1125.0,1929.0,0.0,0.263261,-0.602442
2013-09-16 13:58:00,2013-09-16 13:58:00,3358.5,3359.0,3358.75,304.0,2133.0,0.0,0.750513,0.263261


## Let's try to fit Logistic Regression

In [5]:
def convert_scores(df, column):
    scores = []
    for i, row in df.iterrows():
        scores.append(np.mean(row[column]))
    return scores
scores_columns = ['f1', 'kappa', 'matthews', 'precision', 'recall', 'roc_auc', 'train_f1', 'train_kappa',
       'train_matthews', 'train_precision', 'train_recall', 'train_roc_auc']

In [6]:
df_result = pd.DataFrame()
features = ['queue_imbalance', 'prev_queue_imbalance']
res = model.validate_model(LogisticRegressionCV(), 
                           df[features], df['mid_price_indicator'])
res['features'] = ', '.join(features)
res['method'] = 'logistic'
df_result = df_result.append(res, ignore_index=True)

features = ['queue_imbalance']
res = model.validate_model(LogisticRegressionCV(), 
                           df[features], df['mid_price_indicator'])
res['features'] = ', '.join(features)
res['method'] = 'logistic'
df_result = df_result.append(res, ignore_index=True)

for col in scores_columns:
    df_result[col] = convert_scores(df_result, col)
df_result

,f1,features,kappa,matthews,method,precision,recall,roc_auc,train_f1,train_kappa,train_matthews,train_precision,train_recall,train_roc_auc
0,0.568289,"queue_imbalance, prev_queue_imbalance",0.102187,0.103514,logistic,0.558478,0.583579,0.551004,0.568510,0.108828,0.110824,0.550162,0.594969,0.554291
1,0.571894,queue_imbalance,0.106583,0.108163,logistic,0.559988,0.589789,0.553134,0.569199,0.108485,0.110602,0.549828,0.597351,0.554113


In [7]:
gammas = [0.01, 0.1, 1, 10, 100, 1000]
cs = [0.01, 0.1, 1, 10, 100, 1000]
coef0s = [0.1, 1, 10, 100]

for g in gammas:
    for c in cs:
        for coef in coef0s:
            clf = SVC(kernel='sigmoid', gamma=g, C=c, coef0=coef)
            features = ['queue_imbalance', 'prev_queue_imbalance']
            res = model.validate_model(clf, df[features], df['mid_price_indicator'])
            res['features'] = ', '.join(features) 
            res['method'] = 'svm_sigmoid'
            res['gamma'] = g
            res['coef0'] = coef
            res['c'] = c
            df_result = df_result.append(res, ignore_index=True)

            clf = SVC(kernel='sigmoid', gamma=g, C=c, coef0=coef)
            features = ['queue_imbalance']
            res = model.validate_model(clf, df[features], df['mid_price_indicator'])
            res['features'] = ', '.join(features) 
            res['method'] = 'svm_sigmoid'
            res['gamma'] = g
            res['coef0'] = coef
            res['c'] = c
            df_result = df_result.append(res, ignore_index=True)
df_result

,f1,features,kappa,matthews,method,precision,recall,roc_auc,train_f1,train_kappa,train_matthews,train_precision,train_recall,train_roc_auc,c,coef0,gamma
0,0.568289,"queue_imbalance, prev_queue_imbalance",0.102187,0.103514,logistic,0.558478,0.583579,0.551004,0.56851,0.108828,0.110824,0.550162,0.594969,0.554291,NaN,NaN,NaN
1,0.571894,queue_imbalance,0.106583,0.108163,logistic,0.559988,0.589789,0.553134,0.569199,0.108485,0.110602,0.549828,0.597351,0.554113,NaN,NaN,NaN
2,"[0.0, 0.0, 0.0, 0.0, 0.6795454545454546, 0.700...","queue_imbalance, prev_queue_imbalance","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.5146299483648882, 0.538...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0.0, 0.0, 0.0, 0.0, 0.6674311926605505, 0.669...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.5008605851979346, 0.503...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]",0.01,0.1,0.01
3,"[0.0, 0.0, 0.0, 0.0, 0.6795454545454546, 0.700...",queue_imbalance,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.5146299483648882, 0.538...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0.0, 0.0, 0.0, 0.0, 0.6674311926605505, 0.669...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.5008605851979346, 0.503...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]",0.01,0.1,0.01
4,"[0.0, 0.0, 0.0, 0.0, 0.6795454545454546, 0.700...","queue_imbalance, prev_queue_imbalance","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.5146299483648882, 0.538...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0.0, 0.0, 0.0, 0.0, 0.6674311926605505, 0.669...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.5008605851979346, 0.503...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]",0.01,1.0,0.01
5,"[0.0, 0.0, 0.0, 0.0, 0.6795454545454546, 0.700...",queue_imbalance,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.5146299483648882, 0.538...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0.0, 0.0, 0.0, 0.0, 0.6674311926605505, 0.669...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.5008605851979346, 0.503...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]",0.01,1.0,0.01
6,"[0.0, 0.0, 0.0, 0.0, 0.6795454545454546, 0.700...","queue_imbalance, prev_queue_imbalance","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.5146299483648882, 0.538...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0.0, 0.0, 0.0, 0.0, 0.6674311926605505, 0.669...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.5008605851979346, 0.503...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]",0.01,10.0,0.01
7,"[0.0, 0.0, 0.0, 0.0, 0.6795454545454546, 0.700...",queue_imbalance,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.5146299483648882, 0.538...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]"

In [8]:
gammas = [0.01, 0.1, 1, 10, 100, 1000]
cs = [0.01, 0.1, 1, 10, 100, 1000]

for g in gammas:
    for c in cs:
        clf = SVC(kernel='rbf', gamma=g, C=c)
        features = ['queue_imbalance', 'prev_queue_imbalance']
        res = model.validate_model(clf, df[features], df['mid_price_indicator'])
        res['features'] = ', '.join(features) 
        res['method'] = 'svm_rbf'
        res['gamma'] = g
        res['c'] = c
        df_result = df_result.append(res, ignore_index=True)

        clf = SVC(kernel='sigmoid', gamma=g, C=c)
        features = ['queue_imbalance']
        res = model.validate_model(clf, df[features], df['mid_price_indicator'])
        res['features'] = ', '.join(features) 
        res['method'] = 'svm_rbf'
        res['gamma'] = g
        res['c'] = c
        df_result = df_result.append(res, ignore_index=True)
df_result

,f1,features,kappa,matthews,method,precision,recall,roc_auc,train_f1,train_kappa,train_matthews,train_precision,train_recall,train_roc_auc,c,coef0,gamma
0,0.568289,"queue_imbalance, prev_queue_imbalance",0.102187,0.103514,logistic,0.558478,0.583579,0.551004,0.56851,0.108828,0.110824,0.550162,0.594969,0.554291,NaN,NaN,NaN
1,0.571894,queue_imbalance,0.106583,0.108163,logistic,0.559988,0.589789,0.553134,0.569199,0.108485,0.110602,0.549828,0.597351,0.554113,NaN,NaN,NaN
2,"[0.0, 0.0, 0.0, 0.0, 0.6795454545454546, 0.700...","queue_imbalance, prev_queue_imbalance","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.5146299483648882, 0.538...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0.0, 0.0, 0.0, 0.0, 0.6674311926605505, 0.669...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.5008605851979346, 0.503...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]",0.01,0.1,0.01
3,"[0.0, 0.0, 0.0, 0.0, 0.6795454545454546, 0.700...",queue_imbalance,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.5146299483648882, 0.538...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0.0, 0.0, 0.0, 0.0, 0.6674311926605505, 0.669...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.5008605851979346, 0.503...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]",0.01,0.1,0.01
4,"[0.0, 0.0, 0.0, 0.0, 0.6795454545454546, 0.700...","queue_imbalance, prev_queue_imbalance","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.5146299483648882, 0.538...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0.0, 0.0, 0.0, 0.0, 0.6674311926605505, 0.669...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.5008605851979346, 0.503...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]",0.01,1.0,0.01
5,"[0.0, 0.0, 0.0, 0.0, 0.6795454545454546, 0.700...",queue_imbalance,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.5146299483648882, 0.538...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0.0, 0.0, 0.0, 0.0, 0.6674311926605505, 0.669...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.5008605851979346, 0.503...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]",0.01,1.0,0.01
6,"[0.0, 0.0, 0.0, 0.0, 0.6795454545454546, 0.700...","queue_imbalance, prev_queue_imbalance","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.5146299483648882, 0.538...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0.0, 0.0, 0.0, 0.0, 0.6674311926605505, 0.669...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.5008605851979346, 0.503...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]",0.01,10.0,0.01
7,"[0.0, 0.0, 0.0, 0.0, 0.6795454545454546, 0.700...",queue_imbalance,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.5146299483648882, 0.538...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]"

In [9]:
cs = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

for c in cs:
    clf = SVC(kernel='linear', C=c)
    features = ['queue_imbalance', 'prev_queue_imbalance']
    res = model.validate_model(clf, df[features], df['mid_price_indicator'])
    res['features'] = ', '.join(features) 
    res['method'] = 'svm_linear'
    res['c'] = c
    df_result = df_result.append(res, ignore_index=True)

    clf = SVC(kernel='linear', gamma=g, C=c)
    features = ['queue_imbalance']
    res = model.validate_model(clf, df[features], df['mid_price_indicator'])
    res['features'] = ', '.join(features) 
    res['method'] = 'svm_linear'
    res['c'] = c
    df_result = df_result.append(res, ignore_index=True)
df_result

,f1,features,kappa,matthews,method,precision,recall,roc_auc,train_f1,train_kappa,train_matthews,train_precision,train_recall,train_roc_auc,c,coef0,gamma
0,0.568289,"queue_imbalance, prev_queue_imbalance",0.102187,0.103514,logistic,0.558478,0.583579,0.551004,0.56851,0.108828,0.110824,0.550162,0.594969,0.554291,NaN,NaN,NaN
1,0.571894,queue_imbalance,0.106583,0.108163,logistic,0.559988,0.589789,0.553134,0.569199,0.108485,0.110602,0.549828,0.597351,0.554113,NaN,NaN,NaN
2,"[0.0, 0.0, 0.0, 0.0, 0.6795454545454546, 0.700...","queue_imbalance, prev_queue_imbalance","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.5146299483648882, 0.538...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0.0, 0.0, 0.0, 0.0, 0.6674311926605505, 0.669...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.5008605851979346, 0.503...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]",0.010,0.1,0.01
3,"[0.0, 0.0, 0.0, 0.0, 0.6795454545454546, 0.700...",queue_imbalance,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.5146299483648882, 0.538...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0.0, 0.0, 0.0, 0.0, 0.6674311926605505, 0.669...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.5008605851979346, 0.503...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]",0.010,0.1,0.01
4,"[0.0, 0.0, 0.0, 0.0, 0.6795454545454546, 0.700...","queue_imbalance, prev_queue_imbalance","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.5146299483648882, 0.538...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0.0, 0.0, 0.0, 0.0, 0.6674311926605505, 0.669...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.5008605851979346, 0.503...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]",0.010,1.0,0.01
5,"[0.0, 0.0, 0.0, 0.0, 0.6795454545454546, 0.700...",queue_imbalance,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.5146299483648882, 0.538...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0.0, 0.0, 0.0, 0.0, 0.6674311926605505, 0.669...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.5008605851979346, 0.503...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]",0.010,1.0,0.01
6,"[0.0, 0.0, 0.0, 0.0, 0.6795454545454546, 0.700...","queue_imbalance, prev_queue_imbalance","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.5146299483648882, 0.538...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0.0, 0.0, 0.0, 0.0, 0.6674311926605505, 0.669...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.5008605851979346, 0.503...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]",0.010,10.0,0.01
7,"[0.0, 0.0, 0.0, 0.0, 0.6795454545454546, 0.700...",queue_imbalance,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.5146299483648882, 0.538...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 

In [10]:
df_result.to_csv('res_{}_prev_queue_imbalance.csv'.format(stock))